## 0. تحميل البيانات

In [16]:
import pandas as pd
import numpy as np
from pathlib import Path

# Set up path
BASE_DIR = Path.cwd().parent
PROCESSED_PATH = BASE_DIR / "data" / "processed" / "ride_bookings_clean.csv"

# Load cleaned data
df = pd.read_csv(PROCESSED_PATH)
print("✓ Data loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

✓ Data loaded successfully!
Shape: (150000, 25)

Columns: ['date', 'time', 'booking_id', 'booking_status', 'customer_id', 'vehicle_type', 'pickup_location', 'drop_location', 'avg_vtat', 'avg_ctat', 'cancelled_rides_by_customer', 'reason_for_cancelling_by_customer', 'cancelled_rides_by_driver', 'driver_cancellation_reason', 'incomplete_rides', 'incomplete_rides_reason', 'booking_value', 'ride_distance', 'driver_ratings', 'customer_rating', 'payment_method', 'month', 'day_name', 'is_weekend', 'hour']


---
## 2.1 تحليل الحجوزات (Analyse des réservations)

### ماذا سنفعل؟
حساب العدد الإجمالي للحجوزات وفهم توزيعها حسب الحالة

### لماذا هذا مهم؟
يعطينا نظرة عامة على حجم الأعمال والمؤشرات الأساسية

### ما هو الناتج المتوقع؟
جدول يوضح عدد الحجوزات حسب الحالة (مكتملة، ملغاة، إلخ)

In [17]:
# KPI 1: Total Bookings
total_bookings = len(df)
print(f"📊 إجمالي الحجوزات: {total_bookings:,}")

# Distribution by Booking Status
booking_status_dist = df["booking_status"].value_counts().sort_values(ascending=False)
print("\n📋 توزيع الحجوزات حسب الحالة:")
print(booking_status_dist)
print("\n📈 النسب المئوية:")
print((booking_status_dist / total_bookings * 100).round(2))

📊 إجمالي الحجوزات: 150,000

📋 توزيع الحجوزات حسب الحالة:
booking_status
COMPLETED                93000
CANCELLED BY DRIVER      27000
NO DRIVER FOUND          10500
CANCELLED BY CUSTOMER    10500
INCOMPLETE                9000
Name: count, dtype: int64

📈 النسب المئوية:
booking_status
COMPLETED                62.0
CANCELLED BY DRIVER      18.0
NO DRIVER FOUND           7.0
CANCELLED BY CUSTOMER     7.0
INCOMPLETE                6.0
Name: count, dtype: float64


### ماذا سنفعل؟
تحليل توزيع أنواع المركبات والطرق المفضلة للدفع

### لماذا هذا مهم؟
يساعد في فهم تفضيلات العملاء والمنتجات الرابحة

### ما هو الناتج المتوقع؟
جداول توضح عدد الحجوزات لكل نوع مركبة وطريقة دفع

In [18]:
# Vehicle Type Distribution
vehicle_type_dist = df["vehicle_type"].value_counts().sort_values(ascending=False)
print("🚗 توزيع أنواع المركبات:")
print(vehicle_type_dist)
print(f"\nإجمالي أنواع المركبات: {vehicle_type_dist.nunique()}")

🚗 توزيع أنواع المركبات:
vehicle_type
AUTO             37419
GO MINI          29806
GO SEDAN         27141
BIKE             22517
PREMIER SEDAN    18111
EBIKE            10557
UBER XL           4449
Name: count, dtype: int64

إجمالي أنواع المركبات: 7


In [25]:
# Payment Method Distribution
if "payment_method" in df.columns:
    payment_method_dist = df["payment_method"].value_counts().sort_values(ascending=False)
    print("💳 توزيع طرق الدفع:")
    print(payment_method_dist)
    print(f"\nإجمالي طرق الدفع: {payment_method_dist.nunique()}")
else:
    print("⚠️ عمود payment_method غير موجود")

💳 توزيع طرق الدفع:
payment_method
NAN            48000
UPI            45909
CASH           25367
UBER WALLET    12276
CREDIT CARD    10209
DEBIT CARD      8239
Name: count, dtype: int64

إجمالي طرق الدفع: 6


### 📊 رؤى من تحليل الحجوزات (Insights)
• أكثر أنواع المركبات طلباً وأقلها
• طريقة الدفع الأكثر استخداماً من قبل العملاء
• نسبة الحجوزات المكتملة مقابل الملغاة

---
## 2.2 التحليل الزمني (Analyse temporelle)

### ماذا سنفعل؟
تحليل تطور الحجوزات عبر الأيام والأشهر والساعات

### لماذا هذا مهم؟
يساعد في فهم أوقات الذروة واتجاهات الطلب

### ما هو الناتج المتوقع؟
جداول وإحصائيات عن الحجوزات في ساعات وأيام مختلفة

In [26]:
# Peak Hours Analysis
if "hour" in df.columns:
    rides_per_hour = df["hour"].value_counts().sort_index()
    print("⏰ الحجوزات حسب الساعة (ساعات الذروة):")
    print(rides_per_hour)
    
    peak_hour = rides_per_hour.idxmax()
    print(f"\n🔥 ساعة الذروة: الساعة {int(peak_hour)}:00 مع {rides_per_hour.max()} حجز")
else:
    print("⚠️ عمود hour غير موجود")

⏰ الحجوزات حسب الساعة (ساعات الذروة):
hour
0      1373
1      1360
2      1339
3      1383
4      1321
5      2786
6      4160
7      5450
8      6861
9      8234
10     9577
11     8390
12     7006
13     5470
14     7031
15     8202
16     9633
17    11044
18    12397
19    11047
20     9630
21     8103
22     5441
23     2762
Name: count, dtype: int64

🔥 ساعة الذروة: الساعة 18:00 مع 12397 حجز


In [21]:
# Weekday vs Weekend Analysis
if "is_weekend" in df.columns:
    weekend_analysis = df["is_weekend"].value_counts()
    print("📅 أيام الأسبوع مقابل نهاية الأسبوع:")
    print(f"أيام العمل (الأحد-الخميس): {weekend_analysis[False]:,} حجز")
    print(f"نهاية الأسبوع (الجمعة-السبت): {weekend_analysis[True]:,} حجز")
    
    weekend_pct = (weekend_analysis[True] / total_bookings) * 100
    print(f"\nنسبة حجوزات نهاية الأسبوع: {weekend_pct:.2f}%")
else:
    print("⚠️ عمود is_weekend غير موجود")

📅 أيام الأسبوع مقابل نهاية الأسبوع:
أيام العمل (الأحد-الخميس): 107,060 حجز
نهاية الأسبوع (الجمعة-السبت): 42,940 حجز

نسبة حجوزات نهاية الأسبوع: 28.63%


In [22]:
# Rides per Day Name
if "day_name" in df.columns:
    rides_per_day = df["day_name"].value_counts()
    day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    rides_per_day = rides_per_day.reindex([d for d in day_order if d in rides_per_day.index])
    
    print("📊 الحجوزات حسب اليوم من الأسبوع:")
    print(rides_per_day)
else:
    print("⚠️ عمود day_name غير موجود")

📊 الحجوزات حسب اليوم من الأسبوع:
day_name
Monday       21644
Tuesday      21391
Wednesday    21413
Thursday     21215
Friday       21397
Saturday     21542
Sunday       21398
Name: count, dtype: int64


In [23]:
# Monthly Evolution
if "month" in df.columns:
    rides_per_month = df["month"].value_counts().sort_index()
    print("📈 تطور الحجوزات حسب الشهر:")
    print(rides_per_month)
    
    # Month names in Arabic
    month_names_ar = {
        1: "يناير", 2: "فبراير", 3: "مارس", 4: "أبريل",
        5: "مايو", 6: "يونيو", 7: "يوليو", 8: "أغسطس",
        9: "سبتمبر", 10: "أكتوبر", 11: "نوفمبر", 12: "ديسمبر"
    }
    print("\n📅 بالأسماء:")
    for month, count in rides_per_month.items():
        print(f"{month_names_ar.get(month, '')}: {count:,}")
else:
    print("⚠️ عمود month غير موجود")

📈 تطور الحجوزات حسب الشهر:
month
1     12861
2     11927
3     12719
4     12199
5     12778
6     12440
7     12897
8     12636
9     12248
10    12651
11    12394
12    12250
Name: count, dtype: int64

📅 بالأسماء:
يناير: 12,861
فبراير: 11,927
مارس: 12,719
أبريل: 12,199
مايو: 12,778
يونيو: 12,440
يوليو: 12,897
أغسطس: 12,636
سبتمبر: 12,248
أكتوبر: 12,651
نوفمبر: 12,394
ديسمبر: 12,250


### 📊 رؤى من التحليل الزمني
• ساعات الذروة والساعات منخفضة الطلب
• الفرق بين حجوزات أيام العمل ونهاية الأسبوع
• الاتجاهات الموسمية في عدد الحجوزات

---
## 2.3 تحليل الإلغاءات (Analyse des annulations)

### ماذا سنفعل؟
حساب معدلات الإلغاء ومقارنة إلغاءات العملاء بإلغاءات السائقين

### لماذا هذا مهم؟
الإلغاءات تؤثر على الإيرادات والرضا وكفاءة النظام

### ما هو الناتج المتوقع؟
إحصائيات عن معدل الإلغاء والأسباب الرئيسية

In [24]:
# Cancellation Rate
completed_rides = len(df[df["booking_status"] == "Completed"])
success_rate = (completed_rides / total_bookings) * 100
cancellation_rate = 100 - success_rate

print(f"✅ الحجوزات المكتملة: {completed_rides:,} ({success_rate:.2f}%)")
print(f"❌ معدل الإلغاء: {cancellation_rate:.2f}%")

# Cancelled rides breakdown
cancelled_df = df[df["booking_status"] != "Completed"]
print(f"\n📊 إجمالي الحجوزات الملغاة: {len(cancelled_df):,}")

✅ الحجوزات المكتملة: 0 (0.00%)
❌ معدل الإلغاء: 100.00%

📊 إجمالي الحجوزات الملغاة: 150,000


In [ ]:
# Customer vs Driver Cancellations
if "cancelled_rides_by_customer" in df.columns and "cancelled_rides_by_driver" in df.columns:
    customer_cancellations = df["cancelled_rides_by_customer"].sum()
    driver_cancellations = df["cancelled_rides_by_driver"].sum()
    
    print("🛑 مقارنة الإلغاءات:")
    print(f"إلغاءات العملاء: {int(customer_cancellations):,}")
    print(f"إلغاءات السائقين: {int(driver_cancellations):,}")
    
    if (customer_cancellations + driver_cancellations) > 0:
        customer_pct = (customer_cancellations / (customer_cancellations + driver_cancellations)) * 100
        print(f"\nنسبة إلغاءات العملاء: {customer_pct:.2f}%")
        print(f"نسبة إلغاءات السائقين: {100-customer_pct:.2f}%")
else:
    print("⚠️ أعمدة الإلغاء غير موجودة")

In [ ]:
# Top Cancellation Reasons - Customer
if "reason_for_cancelling_by_customer" in df.columns:
    customer_reasons = df[df["reason_for_cancelling_by_customer"].notna()]["reason_for_cancelling_by_customer"].value_counts().head(10)
    print("❌ أكثر 10 أسباب لإلغاء العملاء:")
    print(customer_reasons)
else:
    print("⚠️ عمود reason_for_cancelling_by_customer غير موجود")

In [ ]:
# Top Cancellation Reasons - Driver
if "driver_cancellation_reason" in df.columns:
    driver_reasons = df[df["driver_cancellation_reason"].notna()]["driver_cancellation_reason"].value_counts().head(10)
    print("❌ أكثر 10 أسباب لإلغاء السائقين:")
    print(driver_reasons)
else:
    print("⚠️ عمود driver_cancellation_reason غير موجود")

In [ ]:
# Impact of Cancellations on Revenue
if "booking_value" in df.columns:
    completed_revenue = df[df["booking_status"] == "Completed"]["booking_value"].sum()
    cancelled_revenue_lost = df[df["booking_status"] != "Completed"]["booking_value"].sum()
    
    print("💰 تأثير الإلغاءات على الإيرادات:")
    print(f"إيرادات الحجوزات المكتملة: ${completed_revenue:,.2f}")
    print(f"الإيرادات المفقودة (بسبب الإلغاءات): ${cancelled_revenue_lost:,.2f}")
    
    total_potential_revenue = completed_revenue + cancelled_revenue_lost
    lost_pct = (cancelled_revenue_lost / total_potential_revenue) * 100 if total_potential_revenue > 0 else 0
    print(f"نسبة الإيرادات المفقودة: {lost_pct:.2f}%")
else:
    print("⚠️ عمود booking_value غير موجود")

### 📊 رؤى من تحليل الإلغاءات
• معدل الإلغاء الإجمالي والعامل الأكثر تأثيراً
• من يلغي أكثر: العملاء أم السائقين؟
• الأسباب الرئيسية للإلغاء والإيرادات المفقودة

---
## 2.4 التحليل المالي (Analyse financière)

### ماذا سنفعل؟
تحليل الإيرادات الإجمالية والمتوسط والتوزيع حسب الفئات المختلفة

### لماذا هذا مهم؟
فهم أداء الأعمال المالية وتحديد الفرص الرابحة

### ما هو الناتج المتوقع؟
جداول وإحصائيات تفصيلية عن الإيرادات

In [ ]:
# Total Revenue (completed rides only)
if "booking_value" in df.columns:
    total_revenue = df[df["booking_status"] == "Completed"]["booking_value"].sum()
    avg_revenue_per_ride = df[df["booking_status"] == "Completed"]["booking_value"].mean()
    
    print(f"💰 إجمالي الإيرادات (الحجوزات المكتملة): ${total_revenue:,.2f}")
    print(f"💵 متوسط الإيرادات لكل حجز: ${avg_revenue_per_ride:.2f}")
    print(f"📊 الحد الأدنى: ${df[df['booking_status'] == 'Completed']['booking_value'].min():.2f}")
    print(f"📊 الحد الأقصى: ${df[df['booking_status'] == 'Completed']['booking_value'].max():.2f}")
else:
    print("⚠️ عمود booking_value غير موجود")

In [ ]:
# Revenue by Vehicle Type
if "booking_value" in df.columns and "vehicle_type" in df.columns:
    completed_df = df[df["booking_status"] == "Completed"]
    revenue_by_vehicle = completed_df.groupby("vehicle_type")["booking_value"].agg(["sum", "mean", "count"])
    revenue_by_vehicle.columns = ["Total Revenue", "Avg per Ride", "Ride Count"]
    revenue_by_vehicle = revenue_by_vehicle.sort_values("Total Revenue", ascending=False)
    
    print("🚗 الإيرادات حسب نوع المركبة:")
    print(revenue_by_vehicle)
else:
    print("⚠️ أعمدة مفقودة")

In [ ]:
# Revenue by Payment Method
if "booking_value" in df.columns and "payment_method" in df.columns:
    completed_df = df[df["booking_status"] == "Completed"]
    revenue_by_payment = completed_df.groupby("payment_method")["booking_value"].agg(["sum", "mean", "count"])
    revenue_by_payment.columns = ["Total Revenue", "Avg per Ride", "Ride Count"]
    revenue_by_payment = revenue_by_payment.sort_values("Total Revenue", ascending=False)
    
    print("💳 الإيرادات حسب طريقة الدفع:")
    print(revenue_by_payment)
else:
    print("⚠️ أعمدة مفقودة")

In [ ]:
# Correlation: Distance vs Booking Value
if "ride_distance" in df.columns and "booking_value" in df.columns:
    completed_df = df[df["booking_status"] == "Completed"].dropna(subset=["ride_distance", "booking_value"])
    
    if len(completed_df) > 0:
        correlation = completed_df["ride_distance"].corr(completed_df["booking_value"])
        print(f"📊 الارتباط بين المسافة وقيمة الحجز: {correlation:.4f}")
        print(f"\n📈 إحصائيات المسافة (المسافات بالكم):")
        print(f"متوسط المسافة: {completed_df['ride_distance'].mean():.2f} كم")
        print(f"الحد الأدنى: {completed_df['ride_distance'].min():.2f} كم")
        print(f"الحد الأقصى: {completed_df['ride_distance'].max():.2f} كم")
    else:
        print("لا توجد بيانات كافية")
else:
    print("⚠️ أعمدة ride_distance أو booking_value غير موجودة")

### 📊 رؤى من التحليل المالي
• إجمالي الإيرادات ومتوسط الإيرادات لكل حجز
• أنواع المركبات والطرق الأكثر رب حية
• قوة الارتباط بين المسافة وقيمة الحجز

---
## 2.5 تحليل الرضا (Analyse de la satisfaction)

### ماذا سنفعل؟
تحليل تقييمات العملاء والسائقين والعوامل المؤثرة عليها

### لماذا هذا مهم؟
رضا العملاء يؤثر على الاحتفاظ بهم والنمو المستقبلي

### ما هو الناتج المتوقع؟
إحصائيات عن التقييمات والعلاقات بين التقييمات والمتغيرات الأخرى

In [ ]:
# Average Ratings
if "customer_rating" in df.columns:
    completed_with_rating = df[(df["booking_status"] == "Completed") & (df["customer_rating"].notna())]
    
    if len(completed_with_rating) > 0:
        avg_customer_rating = completed_with_rating["customer_rating"].mean()
        print(f"⭐ متوسط تقييم العملاء: {avg_customer_rating:.2f}/5.0")
        print(f"📊 عدد التقييمات: {len(completed_with_rating):,}")
        print(f"📊 الحد الأدنى: {completed_with_rating['customer_rating'].min():.1f}")
        print(f"📊 الحد الأقصى: {completed_with_rating['customer_rating'].max():.1f}")
    else:
        print("لا توجد تقييمات")
else:
    print("⚠️ عمود customer_rating غير موجود")

In [ ]:
# Driver Ratings
if "driver_ratings" in df.columns:
    completed_with_driver_rating = df[(df["booking_status"] == "Completed") & (df["driver_ratings"].notna())]
    
    if len(completed_with_driver_rating) > 0:
        avg_driver_rating = completed_with_driver_rating["driver_ratings"].mean()
        print(f"⭐ متوسط تقييم السائقين: {avg_driver_rating:.2f}/5.0")
        print(f"📊 عدد التقييمات: {len(completed_with_driver_rating):,}")
        print(f"📊 الحد الأدنى: {completed_with_driver_rating['driver_ratings'].min():.1f}")
        print(f"📊 الحد الأقصى: {completed_with_driver_rating['driver_ratings'].max():.1f}")
    else:
        print("لا توجد تقييمات")
else:
    print("⚠️ عمود driver_ratings غير موجود")

In [ ]:
# Customer Rating by Vehicle Type
if "customer_rating" in df.columns and "vehicle_type" in df.columns:
    completed_with_rating = df[(df["booking_status"] == "Completed") & (df["customer_rating"].notna())]
    rating_by_vehicle = completed_with_rating.groupby("vehicle_type")["customer_rating"].agg(["mean", "count"])
    rating_by_vehicle.columns = ["Avg Rating", "Count"]
    rating_by_vehicle = rating_by_vehicle.sort_values("Avg Rating", ascending=False)
    
    print("⭐ متوسط تقييمات العملاء حسب نوع المركبة:")
    print(rating_by_vehicle)
else:
    print("⚠️ أعمدة مفقودة")

In [ ]:
# Wait Time vs Rating (if available)
if "avg_vtat" in df.columns and "customer_rating" in df.columns:
    completed_with_data = df[(df["booking_status"] == "Completed") & (df["customer_rating"].notna()) & (df["avg_vtat"].notna())]
    
    if len(completed_with_data) > 0:
        correlation_wait_rating = completed_with_data["avg_vtat"].corr(completed_with_data["customer_rating"])
        print(f"📊 الارتباط بين وقت الانتظار والتقييم: {correlation_wait_rating:.4f}")
        print(f"⏱️ متوسط وقت الانتظار: {completed_with_data['avg_vtat'].mean():.2f} دقيقة")
    else:
        print("لا توجد بيانات كافية")
else:
    print("⚠️ أعمدة avg_vtat أو customer_rating غير موجودة")

In [ ]:
# Distance vs Rating
if "ride_distance" in df.columns and "customer_rating" in df.columns:
    completed_with_data = df[(df["booking_status"] == "Completed") & (df["customer_rating"].notna()) & (df["ride_distance"].notna())]
    
    if len(completed_with_data) > 0:
        correlation_distance_rating = completed_with_data["ride_distance"].corr(completed_with_data["customer_rating"])
        print(f"📊 الارتباط بين المسافة والتقييم: {correlation_distance_rating:.4f}")
        print(f"🚗 متوسط مسافة الرحلة: {completed_with_data['ride_distance'].mean():.2f} كم")
    else:
        print("لا توجد بيانات كافية")
else:
    print("⚠️ أعمدة ride_distance أو customer_rating غير موجودة")

### 📊 رؤى من تحليل الرضا
• متوسط تقييمات العملاء والسائقين بشكل عام
• أنواع المركبات الأكثر رضا من العملاء
• تأثير وقت الانتظار والمسافة على التقييمات

---
## الخلاصة (Résumé)
انتهينا من تحليل شامل للبيانات غطى:
- تحليل الحجوزات والتوزيعات
- الأنماط الزمنية والساعات الذروة
- مشاكل الإلغاء والإيرادات المفقودة
- الأداء المالي الشامل
- رضا العملاء والعوامل المؤثرة عليها